# Shadow Finder ☀️🌐

A tool to estimate the points on the Earth's surface where a shadow of a particular length could occur, for geolocation purposes.

Using an object's height, the lenth of its shadow, the date and the time, this code estimates the possible locations of that shadow.

> <font color='#ffc107'>Important:</font> The shadow length must be measured at right angles to the object 📐 This means that you might have to correct for the perspective of an image before using this tool. 

In [ ]:
# @title Find a Shadow 🔎 { display-mode: "form" }

# @markdown ### ⬅️ Click to find possible locations that match the below information (takes around 10 - 20 seconds)

# @markdown Object and shadow are measured at right angles in arbitrary units
object_height = 10 # @param {type:"number"} Height of object in arbitrary units
shadow_length = 8 # @param {type:"number"} Length of shadow in arbitrary units

# @markdown Date and time must be given in UTC, using the time format hh:mm:ss
date = '2024-02-29' # @param {type:"date"}
time = '12:00:00' # @param {type:"string"}

#Create output files
output = f"./shadowfinder_{object_height}_{shadow_length}_{date}T{time}.png"
logfile = f"./shadowfinder_{object_height}_{shadow_length}_{date}T{time}.log"

# Imports
![ ! -f "deps_loaded" ] & pip install suncalc basemap >> {logfile} 2>&1 & touch deps_loaded

from suncalc import get_position
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.basemap import Basemap


datetime_date = datetime.datetime.strptime(date, "%Y-%m-%d").date()
datetime_time = datetime.datetime.strptime(time, "%H:%M:%S").time()
#datetime_time = datetime.time(hours, minutes, seconds)
date_time = datetime.datetime.combine(datetime_date, datetime_time, tzinfo=datetime.timezone.utc) # Date and time of interest


# Main calculation
# Evaluate the sun's  length at a grid of points on the Earth's surface
lats = np.arange(-90, 90, 0.25)
lons = np.arange(-180, 180, 0.25)

lons, lats = np.meshgrid(lons, lats)

pos_obj = get_position(date_time, lons, lats)
sun_altitude = pos_obj['altitude'] # in radians

# Calculate the shadow length
shadow_lengths = object_height / np.apply_along_axis(np.tan, 0, sun_altitude) 

# Replace points where the sun is below the horizon with nan
shadow_lengths[sun_altitude <= 0] = np.nan

# Show the relative difference between the calculated shadow length and the observed shadow length
shadow_relative_length_difference = (shadow_lengths - shadow_length)/shadow_length


# Plotting
fig = plt.figure(figsize=(12, 6))

# Add a simple map of the Earth
m = Basemap(projection='cyl', resolution='c')
m.drawcoastlines()
m.drawcountries()

# Deal with the map projection
x, y = m(lons, lats)

# Set the a color scale and only show the values between 0 and 0.2
cmap = plt.cm.inferno_r
norm = colors.BoundaryNorm(np.arange(0, 0.2, 0.02), cmap.N)

# Plot the data
m.pcolormesh(x, y, np.abs(shadow_relative_length_difference), cmap=cmap, norm=norm,alpha=0.7)

# plt.colorbar(label='Relative Shadow Length Difference')
plt.title(f"Possible Locations at {date_time.strftime('%Y-%m-%d %H:%M:%S %Z')}\n(object height: {object_height}, shadow length: {shadow_length})")
plt.show()
